# Homework 4: Applied ML

We will work with a by-now classic dataset from Robert LaLonde's study "Evaluating the Econometric Evaluations of Training Programs" (1986). The study investigated the effect of a job training program ("National Supported Work Demonstration") on the real earnings of an individual, a couple of years after completion of the program. Your task is to determine the effectiveness of the "treatment" represented by the job training program.

We import all the packages needed here and define plotting functions

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import networkx as nt
import multiprocessing as mp
from scipy.optimize import linear_sum_assignment
from sklearn import linear_model


In [ ]:
def densplot(columns, xlabel, title):
    fig, ax = plt.subplots()
    for i,v in enumerate(columns):
        sns.distplot(v, ax=ax, kde_kws={"label": i})
    plt.title(title)
    plt.xlabel(xlabel, fontsize=12)
    plt.show()
    
def scatplot(xelem, yelem, xlabel, ylabel, title, polyfit=None):
    plt.scatter(xelem, yelem)
    if polyfit:
        plt.plot(np.unique(xelem), np.poly1d(np.polyfit(xelem, yelem, polyfit))(np.unique(xelem)), 'C2')
    plt.title(title)
    plt.xlabel(xlabel, fontsize=12)
    plt.ylabel(ylabel, fontsize=12)
    plt.show()

def boplot(data, title, xlabel, violin): #violin=True means a quantil plot
    if violin:
        sns.violinplot(data=data)
    else:
        sns.boxplot(data=data)
    plt.title(title)
    plt.xlabel(xlabel, fontsize=12)
    plt.show()

We start by importing the data.

#### Dataset description

- `treat`: 1 if the subject participated in the job training program, 0 otherwise
- `age`: the subject's age
- `educ`: years of education
- `race`: categorical variable with three possible values: Black, Hispanic, or White
- `married`: 1 if the subject was married at the time of the training program, 0 otherwise
- `nodegree`: 1 if the subject has earned no school degree, 0 otherwise
- `re74`: real earnings in 1974 (pre-treatment)
- `re75`: real earnings in 1975 (pre-treatment)
- `re78`: real earnings in 1978 (outcome)

In [ ]:
data = pd.read_csv('lalonde.csv', index_col=0)
data.head()

To match with the description and for simplicity in further step, we create the "race" and "white" columns:
(we first check that there are no people that are both black and hispanic in the dataset)

In [ ]:
print("Number of black hispanic people: ", len(data.loc[(data['black']==1) & (data['hispan'] == 1)]))
data["race"] = "white"
data["white"] = 0
#for i,v in enumerate(data['black']):
#    if v == 1:
#        data["race"][i]="black"
#    elif data['hispan'][i] == 1:
#        data["race"][i]="hispanic"
#data

data.loc[data['black'] == 1,'race'] = "black"
data.loc[data['hispan'] == 1,'race'] = "hispanic"
data.loc[(data['black'] == 0) & (data['hispan'] == 0),'white'] = 1
data.head(10)

### 1. A naive analysis

In this part, we compare the distribution of the outcome variable (`re78`) between the two groups (control and test)

In [ ]:
grouped = data.groupby('treat')
grouped0 = grouped.get_group(0)
grouped1 = grouped.get_group(1)

In [ ]:
densplot([grouped0['re78'],grouped1['re78']], "treat", "Comparison of distributions")

**What might a naive "researcher" conclude from this superficial analysis?**
A naive observer would think that the two distributions are pretty similar even though there seems to be more people that didn't follow the program with an income between 1500 and 2000.


In [ ]:
boplot([grouped0['re78'],grouped1['re78']],"Treat comparison boxplots", "treat", False)
boplot([grouped0['re78'],grouped1['re78']],"Treat comparison violin plots", "treat", True)

The boxplot and violin plot (combination of boxplot and kernel density estimate) tend to show the same as we observed previously. Nevertheless, they also show that the group that followed the program contains more outliers, or in other words, more people with (much) higher incomes.
*A naive conclusion would be to say that the program is not very efficient except for a few people that have a very high income*

### 2. A closer look at the data

For each feature in the dataset, we compare its distribution in the treated group with its distribution in the control group. 
Since each feature is pretty different, we cannot plot everything at the same time and we will do each feature at a time.

#### Age

In [ ]:
densplot([grouped0["age"],grouped1["age"]], "age", "Comparison of distributions for age")
boplot([grouped0["age"],grouped1["age"]],"Comparison with boxplots for age", "treat", False)

We observe that even if the median is similar in both datasets according to the age feature, the dataset of people that didn't follow the program (treat=0) contains more older people while the people that followed the program where concentrated between 20 and 30 years old.

*A first observation that we can do, is that the income is often related to the age, as more expericence usually mean a better income. Thus, the fact that the two datasets have different age distribution should be considered in the comparison*

#### Education

In [ ]:
densplot([grouped0["educ"],grouped1["educ"]], "years of education", "Comparison of distributions for education duration")
boplot([grouped0["educ"],grouped1["educ"]],"Comparison with boxplots for education duration", "treat", False)

We see that in terms of education duration, the values are more sparsed for people that didn't follow the program with outliers in both direction (with few or many years of education). Again the median is similar in both datasets but we observe that they are more people that followed the program that received between 10 and 12 years of education while the other dataset is more sparsed with a peak at 13 years of education.


#### Race

We now observe the "race" feature that we built from the "black" and "hispan" features in the beginning

In [ ]:
#TODO: merge both boxplots ?
grouped0['race'].value_counts().plot(kind='bar')
plt.title("Race counts for people that didn't follow the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['race'].value_counts().plot(kind='bar', color='r')
plt.title("Race counts for people that followed the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

In this case, we do not need more plots to see that the people that didn't follow the program were mostly white while the other dataset almost only contains black people.
*Again, it is well known that, sadly, the average income for black people is usually lower than for white people which can also interfer with the results of the study*

#### Married

In [ ]:
#TODO: merge both boxplots ?
grouped0['married'].value_counts().plot(kind='bar')
plt.title("Married counts for people that didn't follow the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['married'].value_counts().plot(kind='bar', color='r')
plt.title("Married counts for people that followed the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

We clearly observe that most of the people that followed the program were single when the married against single counts are balanced in the first dataset.

*The fact that someone is married or not can also have an influence on its income and should be taken into account"

#### Degree

In [ ]:
#TODO: merge both boxplots ?
grouped0['nodegree'].value_counts().plot(kind='bar')
plt.title("Counts for people with/without a degree that didn't follow the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['nodegree'].value_counts().plot(kind='bar', color='r')
plt.title("Counts for people with/without a degree that followed the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

In this case, both datasets are consistent and about two-third of the people do not have a degree in each.

#### real earnings in 1974 (pre-treatment)

In [ ]:
densplot([grouped0["re74"],grouped1["re74"]], "real earnings in 1974", "Comparison of distributions for real earnings in 1974")
boplot([grouped0["re74"],grouped1["re74"]],"Comparison with boxplots for real earnings in 1974", "treat", False)

We observe that both datasets contain a lot of people with no income at this date. We also notice that more people that didn't participate in the program already had a salary at this date which is less the case for people that participated.

#### real earnings in 1975 (pre-treatment)

In [ ]:
densplot([grouped0["re75"],grouped1["re75"]], "real earnings in 1975", "Comparison of distributions for real earnings in 1975")
boplot([grouped0["re75"],grouped1["re75"]],"Comparison with boxplots for real earnings in 1975", "treat", False)

We observe that the income decreased for a lot of people in the first dataset, when it increases for some in the second dataset.

#### Conclusion

We discuss here our main observations and how they relate to our naive observation.
- the dataset of people that didn't followed the program contains more people that are above 30 years old
- most of the people that followed the program were single, which is not the case in the other dataset
- more people already had an income before 1975 in the people that didn't participate in the program

From these observations, we can remark that even though the median values are often similar in both datasets, the distributions are different. Moreover, this is particularly the case for the age and race features which can both have a high impact on the income of a person.


### 3. A propensity score model

We use logistic regression to estimate propensity scores for all points in the dataset.

In [ ]:
logistic = linear_model.LogisticRegression()
feature_cols = ['age', 'educ', 'black', 'hispan', 'white', 'married', 'nodegree', 're74', 're75',] #re78 ??
X = data[feature_cols]
y = data['treat']

propensity = logistic.fit(X, y)
pscore = propensity.predict_proba(X)[:,1] # The predicted propensities by the model
print(pscore[:10])

data['propensity'] = pscore

### 4. Balancing the dataset via matching

We use the propensity scores to match each data point from the treated group with exactly one data point from the control group, while ensuring that each data point from the control group is matched with at most one data point from the treated group.

We start our matching process by creating the matrix of the costs (using the absolute difference between the propensity scores as a cost function) of all the possible assignments. We then use this matrix to solve the linear sum assignment problem (The linear sum assignment problem is also known as minimum weight matching in bipartite graphs).


In [ ]:
group2 = data.groupby('treat')
group20 = group2.get_group(0).reset_index()
group21 = group2.get_group(1).reset_index()

In [ ]:
def col(colone, coltwo):
    return np.array([abs(colone - n) for n in coltwo])

costs = np.array(col(group21['propensity'], group20['propensity']))

print("Computing optimal assigment...", end="")
id_n1, id_n2 = linear_sum_assignment(costs)
sol_costs = costs[id_n1, id_n2]

print("Done: cost of solution = %d" % sol_costs.sum())

match = group21.join(group20.loc[id_n1].reset_index(drop=True), rsuffix="_2")

After the matching, we compare the outcomes (`re78`) between the two groups (treated and control)

In [ ]:
densplot([match["re78"], match["re78_2"]], "real earnings in 1978", "Comparison of distributions for real earnings in 1978")
boplot([match["re78"], match["re78_2"]],"Comparison with boxplots for real earnings in 1978", "treat", False)

As we observed in the beginning, the two datasets have similar distributions even though the group of people that didn't follow the program are still more numerous to have big incomes.

We compare again the feature-value distributions between the two groups but now only for the matched subjects.
In this case, we only show the boxplots and start with all the features that are now well-balanced between the two datasets.

It is clearly the case for:

#### Married

In [ ]:
grouped0['married'].value_counts().plot(kind='bar')
plt.title("OLD Married counts for people that didn't follow the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['married'].value_counts().plot(kind='bar', color='r')
plt.title("OLD Married counts for people that followed the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

match['married_2'].value_counts().plot(kind='bar')
plt.title("Married counts for people that didn't follow the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

match['married'].value_counts().plot(kind='bar', color='r')
plt.title("Married counts for people that followed the program")
plt.xlabel("Married (1) or not (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

#### No Degree

In [ ]:
grouped0['nodegree'].value_counts().plot(kind='bar')
plt.title("OLD Counts for people with/without a degree that didn't follow the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['nodegree'].value_counts().plot(kind='bar', color='r')
plt.title("OLD Counts for people with/without a degree that followed the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

#TODO: merge both boxplots ?
match['nodegree_2'].value_counts().plot(kind='bar')
plt.title("Counts for people with/without a degree that didn't follow the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

match['nodegree'].value_counts().plot(kind='bar', color='r')
plt.title("Counts for people with/without a degree that followed the program")
plt.xlabel("No degree (1) or degree (0)", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

#### real earnings in 1974 (pre-treatment)

In [ ]:
boplot([grouped0["re74"],grouped1["re74"]],"OLD comparison with boxplots for real earnings in 1974", "treat", False)
boplot([match["re74_2"],match["re74"]],"Comparison with boxplots for real earnings in 1974", "treat", False)

#### real earnings in 1975 (pre-treatment)

In [ ]:
boplot([grouped0["re75"],grouped1["re75"]],"OLD comparison with boxplots for real earnings in 1975", "treat", False)
boplot([match["re75_2"],match["re75"]],"Comparison with boxplots for real earnings in 1975", "treat", False)

The remaining features for which the balance is still not completely attaigned are:

#### age

We observe that the remaining people are mostly between 17 and 27 years old in the non-program group and between 20 and 28 in the second one. We will come back to this feature if our next improved matching is still not good enough.

In [ ]:
boplot([grouped0["age"],grouped1["age"]],"OLD Comparison with boxplots for age", "treat", False)
boplot([match["age_2"], match["age"]],"Comparison with boxplots for age", "treat", False)

#### Study Duration

For this feature, the situations is similar than for age. THe two datasets are not perfectly balanced but it might be acceptable. If not, we will come back to this feature later.

In [ ]:
boplot([grouped0["educ"],grouped1["educ"]],"OLD Comparison with boxplots for education duration", "treat", False)
boplot([match["educ_2"],match["educ"]],"Comparison with boxplots for education duration", "treat", False)

#### Race

In [ ]:
grouped0['race'].value_counts().plot(kind='bar')
plt.title("OLD Race counts for people that didn't follow the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

grouped1['race'].value_counts().plot(kind='bar', color='r')
plt.title("OLD Race counts for people that followed the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

match['race_2'].value_counts().plot(kind='bar')
plt.title("Race counts for people that didn't follow the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

match['race'].value_counts().plot(kind='bar', color='r')
plt.title("Race counts for people that followed the program")
plt.xlabel("race", fontsize=12)
plt.ylabel("count", fontsize=12)
plt.show()

**Are you closer to being able to draw valid conclusions now than you were before?**
We are closer than before as the dataset is well-balanced for multiple features. Nevertheless, they are still three features for which the balance is not good enough. 

### 5. Balancing the groups further

We try to see which feature is really a problem:

In [ ]:
matchRace = match.loc[match['race'] == match['race_2']]
print("There are " , len(matchRace), " out of ", len(match), " that are matched with someone of the same race")

matchEduc = match.loc[abs(match['educ'] - match['educ_2'])< int(pd.DataFrame.std(match['educ']))+1]
print("There are " , len(matchEduc), " out of ", len(match), " that are matched with someone that have similar number of years of study (3 or less years of difference)")

matchAge = match.loc[abs(match['age'] - match['age_2'])< int(pd.DataFrame.std(match['age']))+1]
print("There are " , len(matchAge), " out of ", len(match), " that are matched with someone that have similar age (8 or less years of difference)")

matchAll = match.loc[(abs(match['age'] - match['age_2'])< int(pd.DataFrame.std(match['age']))+1) & (abs(match['educ'] - match['educ_2'])< int(pd.DataFrame.std(match['educ']))+1) & (match['race'] == match['race_2'])]
print("There are " , len(matchAll), " out of ", len(match), " that are matched with someone that have similar number of years of study")



### 6. A less naive analysis

In [ ]:
boplot([matchAll['re78_2'],matchAll['re78']],"Treat comparison boxplots matched on All", "treat", False)

boplot([matchAge['re78_2'],matchAge['re78']],"Treat comparison boxplots matched on Age", "treat", False)

boplot([matchEduc['re78_2'],matchEduc['re78']],"Treat comparison boxplots matched on Education duration", "treat", False)

boplot([matchRace['re78_2'],matchRace['re78']],"Treat comparison boxplots matched on Race", "treat", False)



TODO:
drawing and discussion for step 5 and 6
clean code (histograms) and maybe find better way to show the plots
check matching
step 4 or 5 -> redo matching or remove other